In [18]:
import pandas as pd
import numpy as np

df_emission_type = pd.read_csv('co2-fossil-plus-land-use.csv')
df_emission_type = df_emission_type[~((df_emission_type['Code'].isna())|(df_emission_type['Code']=='OWID_WRL'))]
df_emission_type = df_emission_type.rename(columns={
    'Annual CO₂ emissions including land-use change': 'Annual CO2 emissions including land-use change',
    'Annual CO₂ emissions from land-use change': 'Annual CO2 emissions from land-use change',
    'Annual CO₂ emissions': 'Annual CO2 emissions'
    })
df_emission_type = df_emission_type[df_emission_type['Year']==max(df_emission_type['Year'])]
df_emission_type

,Entity,Code,Year,Annual CO2 emissions including land-use change,Annual CO2 emissions from land-use change,Annual CO2 emissions
172,Afghanistan,AFG,2022,1.219331e+07,4.536032e+04,12147948.0
691,Albania,ALB,2022,5.058494e+06,1.037645e+05,4954730.0
864,Algeria,DZA,2022,1.782703e+08,1.925176e+06,176345140.0
1037,Andorra,AND,2022,3.653108e+05,-3.334240e+03,368645.0
1210,Angola,AGO,2022,5.326284e+07,3.719257e+07,16070269.0
...,...,...,...,...,...,...
39873,Vietnam,VNM,2022,5.953420e+08,2.517353e+08,343606620.0
39906,Wallis and Futuna,WLF,2022,NaN,NaN,26461.0
40252,Yemen,YEM,2022,9.319126e+06,-2.037221e+06,11356347.0
40425,Zambia,ZMB,2022,6.999363e+07,6.107174e+07,8921889.0


In [19]:
df_mapping_region = pd.read_csv('country-region-mapping.csv')
df_emission_type = pd.merge(left=df_emission_type, right=df_mapping_region, how='left')
df_emission_type

,Entity,Code,Year,Annual CO2 emissions including land-use change,Annual CO2 emissions from land-use change,Annual CO2 emissions,Region
0,Afghanistan,AFG,2022,1.219331e+07,4.536032e+04,12147948.0,Asia
1,Albania,ALB,2022,5.058494e+06,1.037645e+05,4954730.0,Europe
2,Algeria,DZA,2022,1.782703e+08,1.925176e+06,176345140.0,Africa
3,Andorra,AND,2022,3.653108e+05,-3.334240e+03,368645.0,Europe
4,Angola,AGO,2022,5.326284e+07,3.719257e+07,16070269.0,Africa
...,...,...,...,...,...,...,...
212,Vietnam,VNM,2022,5.953420e+08,2.517353e+08,343606620.0,Asia
213,Wallis and Futuna,WLF,2022,NaN,NaN,26461.0,Oceania
214,Yemen,YEM,2022,9.319126e+06,-2.037221e+06,11356347.0,Asia
215,Zambia,ZMB,2022,6.999363e+07,6.107174e+07,8921889.0,Africa


In [20]:
top_n = 3
df_emission_type['Rank'] = df_emission_type.groupby('Region')['Annual CO2 emissions including land-use change'].rank(method='first', ascending=False)
df_emission_type = df_emission_type[df_emission_type['Rank']<=top_n].reset_index(drop=True)
for c in ['Annual CO2 emissions from land-use change', 'Annual CO2 emissions']:
    df_emission_type[c+' ABS'] = np.abs(df_emission_type[c])
df_emission_type['Annual CO2 emissions including land-use change'] = df_emission_type['Annual CO2 emissions from land-use change']+df_emission_type['Annual CO2 emissions']
df_emission_type['Annual CO2 emissions including land-use change ABS'] = df_emission_type['Annual CO2 emissions from land-use change ABS']+df_emission_type['Annual CO2 emissions ABS']
df_emission_type_region = df_emission_type[['Region', 'Annual CO2 emissions including land-use change ABS', 'Annual CO2 emissions from land-use change ABS', 'Annual CO2 emissions ABS', 'Annual CO2 emissions including land-use change', 'Annual CO2 emissions from land-use change', 'Annual CO2 emissions']].groupby('Region').sum().reset_index().rename(columns={'Annual CO2 emissions including land-use change ABS': 'Annual region CO2 emission ABS', 'Annual CO2 emissions from land-use change ABS': 'region land ABS', 'Annual CO2 emissions ABS': 'region fossil ABS', 'Annual CO2 emissions including land-use change': 'Annual region CO2 emission', 'Annual CO2 emissions from land-use change': 'region land', 'Annual CO2 emissions': 'region fossil'})
df_emission_type = pd.merge(left=df_emission_type, right=df_emission_type_region, how='left')
df_emission_type = df_emission_type.sort_values(by=['Annual region CO2 emission ABS', 'Annual CO2 emissions including land-use change ABS'], ascending=[False, False]).reset_index(drop=True)
df_emission_type = df_emission_type.rename(columns={'Annual CO2 emissions ABS': 'fossil ABS', 'Annual CO2 emissions from land-use change ABS': 'land ABS', 'Annual CO2 emissions': 'fossil', 'Annual CO2 emissions from land-use change': 'land'})
for c in ['Annual CO2 emissions including land-use change', 'fossil', 'land', 'Annual region CO2 emission', 'region fossil', 'region land']:
    df_emission_type[c] /= 1_000_000
    df_emission_type[c+' ABS'] /= 1_000_000
    df_emission_type[c] = df_emission_type[c].round(0)
    df_emission_type[c+' ABS'] = df_emission_type[c+' ABS'].round(0)
    df_emission_type[c] = df_emission_type[c].astype(int)
    df_emission_type[c+' ABS'] = df_emission_type[c+' ABS'].astype(int)
df_emission_type

,Entity,Code,Year,Annual CO2 emissions including land-use change,land,fossil,Region,Rank,land ABS,fossil ABS,Annual CO2 emissions including land-use change ABS,Annual region CO2 emission ABS,region land ABS,region fossil ABS,Annual region CO2 emission,region land,region fossil
0,China,CHN,2022,12378,982,11397,Asia,1.0,982,11397,12378,16890,1935,14955,16890,1935,14955
1,India,IND,2022,2864,34,2830,Asia,2.0,34,2830,2864,16890,1935,14955,16890,1935,14955
2,Indonesia,IDN,2022,1647,919,729,Asia,3.0,919,729,1647,16890,1935,14955,16890,1935,14955
3,United States,USA,2022,5199,142,5057,North America,1.0,142,5057,5199,6728,610,6117,6728,610,6117
4,Mexico,MEX,2022,863,351,512,North America,2.0,351,512,863,6728,610,6117,6728,610,6117
5,Canada,CAN,2022,666,118,548,North America,3.0,118,548,666,6728,610,6117,6728,610,6117
6,Russia,RUS,2022,1965,313,1652,Europe,1.0,313,1652,1965,2973,337,2636,2957,321,2636
7,Germany,DEU,2022,658,-8,666,Europe,2.0,8,666,674,2973,337,2636,2957,321,2636
8,United Kingdom,GBR,2022,335,16,319,Europe,3.0,16,319,335,2973,337,2636,2957,321,2636
9,Brazil,BRA,2022,1596,1113,483,South America,1.0,1113,483,1596,1952,1176,776,1952,1176,776


In [21]:
colors = ['#FFA630', '#D7E8BA', '#4DA1A9', '#2E5077', '#CEABB1', '#E83F6F', '#694A38', '#2C5530']

In [22]:
nodes = []
nodes_region = []
for idx, row in df_emission_type.iterrows():
    nodes.append(pd.DataFrame({'node': [row["Entity"]], 'value': [row['Annual CO2 emissions including land-use change ABS']], 'real_value': [row['Annual CO2 emissions including land-use change']], 'indirect_links': [[idx//top_n+len(df_emission_type)]], 'color': [colors[idx//top_n]]}))
    if idx%top_n==0:
        nodes_region.append(pd.DataFrame({'node': [row["Region"]], 'value': [row['Annual region CO2 emission ABS']], 'real_value': [row['Annual region CO2 emission']], 'indirect_links': [[idx, idx+1, idx+2]], 'color': [colors[idx//top_n]]}))
nodes = pd.concat(nodes+nodes_region+[pd.DataFrame({'node': ['fossil'], 'value': [np.sum(df_emission_type['fossil ABS'])], 'real_value': [np.sum(df_emission_type['fossil'])], 'indirect_links': [[]], 'color': [colors[-2]]}), pd.DataFrame({'node': ['land'], 'value': [np.sum(df_emission_type['land ABS'])], 'real_value': [np.sum(df_emission_type['land'])], 'indirect_links': [[]], 'color': [colors[-1]]})], ignore_index=True)
nodes.to_csv('nodes.csv')
nodes

,node,value,real_value,indirect_links,color
0,China,12378,12378,[18],#FFA630
1,India,2864,2864,[18],#FFA630
2,Indonesia,1647,1647,[18],#FFA630
3,United States,5199,5199,[19],#D7E8BA
4,Mexico,863,863,[19],#D7E8BA
5,Canada,666,666,[19],#D7E8BA
6,Russia,1965,1965,[20],#4DA1A9
7,Germany,674,658,[20],#4DA1A9
8,United Kingdom,335,335,[20],#4DA1A9
9,Brazil,1596,1596,[21],#2E5077


In [23]:
links = []
links_region = []
for idx, row in df_emission_type.iterrows():
    links.append(pd.DataFrame({'source': [idx], 'target': [24], 'value': [row['fossil ABS']], 'real_value': [row['fossil']]}))
    links.append(pd.DataFrame({'source': [idx], 'target': [25], 'value': [row['land ABS']], 'real_value': [row['land']]}))
    if idx%top_n==0:
        links_region.append(pd.DataFrame({'source': [24], 'target': [idx//top_n+len(df_emission_type)], 'value': [row['region fossil ABS']], 'real_value': [row['region fossil']]}))
        links_region.append(pd.DataFrame({'source': [25], 'target': [idx//top_n+len(df_emission_type)], 'value': [row['region land ABS']], 'real_value': [row['region land']]}))

links = pd.concat(links+links_region, ignore_index=True)
links.to_csv('links.csv')
links

,source,target,value,real_value
0,0,24,11397,11397
1,0,25,982,982
2,1,24,2830,2830
3,1,25,34,34
4,2,24,729,729
5,2,25,919,919
6,3,24,5057,5057
7,3,25,142,142
8,4,24,512,512
9,4,25,351,351
